# Scraping stats from hockeyreference.com


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import regex as re
import time
import numpy as np
import json

## User input

In [2]:
# Set the season. Use the 2nd year involved in the season. 
# For example, if the 2021-2022 season is desired, set season = '2022'
season = '2023'

## Read in dict to convert team name to 3 letter code

In [4]:
# Read in team name dictionary for cleaning
with open('../data/team_name_dictionary.txt', 'r') as f:
    # Load the dictionary from the file
    team_name_dict = json.load(f)

## Define function to web scrape season game logs given player_id

In [3]:
def get_game_log(player_id, season):
    try:
        # Create template URL. We will fill in the 2 blanks (___ and ...) with the correct information for each player
        template_url = 'https://www.hockey-reference.com/players.../gamelog/___'

        # Fill in blanks
        url = template_url.replace('...', str(player_id))
        url = url.replace('___', str(season))

        # Get table from hockey reference using pandas
        game_log = pd.read_html(url, attrs = {'class': 'row_summable'})[0]

        # Add the player_id column
        game_log['player_id'] = player_id
        
        # Add the season column
        game_log['season'] = season

        return game_log
    
    except:
        # If we cant find games for a particular player, alert us and save the player_id off to the side. 
        # Then continue on wih the rest.
        return print(player_id, "was not found among season", season, "players on HockeyReference.com")

## Function to get a list of all the player IDs for the season
### This should also create a link between player ID and player name
Now, we want to do this for all players from the user specified season that have played at least 1 game during that season.
It would probably be most accurate to get this list from HockeyReference since that is where we are scraping the player stats.

The url containing these players has the following form and can be modified based on what season the user is interested in. 
* https://www.hockey-reference.com/leagues/NHL_2022_skaters.html
* https://www.hockey-reference.com/leagues/NHL_2022_goalies.html

We will grab the player_id using the href. The names can be retrieved from the same tag (using the text instead of href). The names will be left joined later to the final data frame.

In [4]:
def create_name_to_id_link(season, player_type):
    # Scrape the list of players.
    # Create the url
    season = str(season)
    if player_type == 's':
        names_url = 'https://www.hockey-reference.com/leagues/NHL_' + season + '_skaters.html'
    elif player_type == 'g':
        names_url = 'https://www.hockey-reference.com/leagues/NHL_' + season + '_goalies.html'
    else:
        return 'Invalid player type. Use "s" or "g".'
    
    # Grab the html code 
    response = requests.get(names_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    names_html_list = soup.find_all('td', attrs = {'data-stat':'player'})

    # Here is the list of player names. We can use this to create a dictionary of {'player_id':'player_names'}.
    names = [player_name.text for player_name in names_html_list]
    print('Number of names scraped:', len(names))

    # Grab player ID using the href in <a> tag. Then we don't have to construct them ourselves.
    href_strings = [str(names_html_list[index].a) for index in range(len(names_html_list))]
    ids = [re.search('/\w/\S+[0-9]+', href).group(0) for href in href_strings] # This can also handle names like 'J.T. Compher'

    # This works for getting all of the player IDs in the list
    print('Number of player IDs scraped:', len(ids))

    # Here is the set of unique ones we will use later to iterate through
    unique_ids = list(set(ids))
    unique_ids.sort()
    print('Number of unique player IDs', len(unique_ids), '\n')

    # Create a data frame of player ids and player names. We will use this to join to the final data frame to 
    # add the player names as another column.
    ids_names = pd.DataFrame({'player_id' : ids, 'player_name' : names})
    ids_names = ids_names.drop_duplicates()

    # Return the df/dictionary that links player ID to player name
    return ids_names
        

## Get the game log for each player ID

In [5]:
# Get the names and player ID's of all skaters that played during the season
skater_ids_names = create_name_to_id_link(season = season, player_type = 's')
display(skater_ids_names.head())

Number of names scraped: 1142
Number of player IDs scraped: 1142
Number of unique player IDs 950 



,player_id,player_name
0,/a/abruzni01,Nicholas Abruzzese
1,/a/acciano01,Noel Acciari
4,/a/addisca01,Calen Addison
5,/a/afanaeg01,Egor Afanasyev
6,/a/agozzan01,Andrew Agozzino


In [11]:
# Initalize empty df to store all player game logs combined
skater_games = pd.DataFrame()
unique_skater_ids = skater_ids_names['player_id'].unique()

for i, id in enumerate(unique_skater_ids):
    # Wait some time between each request
    time.sleep(4)
    
    # Get game log for the id
    skater_games = pd.concat([skater_games, get_game_log(id, season = season)], axis = 0)

    # Print our progress as we iterate through all the players
    if i in [25, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]:
        print('Num players:', len(skater_games['player_id'].unique()), '- Num rows in df:', len(skater_games.index))

Num players: 26 - Num rows in df: 1196
Num players: 51 - Num rows in df: 2566
Num players: 101 - Num rows in df: 4971
Num players: 201 - Num rows in df: 9750
Num players: 301 - Num rows in df: 14971
Num players: 401 - Num rows in df: 19863
Num players: 501 - Num rows in df: 25244
Num players: 601 - Num rows in df: 30763
Num players: 701 - Num rows in df: 36387
Num players: 801 - Num rows in df: 41362
Num players: 901 - Num rows in df: 46675


## Get the game log for each goalie ID

In [60]:
# Get the names and player ID's of all goalies that played during the season
goalie_ids_names = create_name_to_id_link(season = season, player_type = 'g')
display(goalie_ids_names.head())

Number of names scraped: 115
Number of player IDs scraped: 115
Number of unique player IDs 107 



,player_id,player_name
0,/a/alexaje01,Jett Alexander
1,/a/allenja01,Jake Allen
2,/a/anderfr01,Frederik Andersen
3,/a/andercr01,Craig Anderson
4,/a/annunju01,Justus Annunen


In [61]:
# Initalize empty df to store all player game logs combined
goalie_games = pd.DataFrame()
unique_goalie_ids = goalie_ids_names['player_id'].unique()

for i, id in enumerate(unique_goalie_ids):
    # Wait some time between each request
    time.sleep(4)
    
    # Get game log for the id
    goalie_games = pd.concat([goalie_games, get_game_log(id, season = season)], axis = 0)

    # Print our progress as we iterate through all the players
    if i in [10, 25, 50, 75, 100, 125]:
        print('Num players:', len(goalie_games['player_id'].unique()), '- Num rows in df:', len(goalie_games.index))

Num players: 11 - Num rows in df: 261
Num players: 26 - Num rows in df: 590
Num players: 51 - Num rows in df: 1311
Num players: 76 - Num rows in df: 1927
Num players: 101 - Num rows in df: 2765


## Clean the data frame of skater statistics

In [14]:
# Rename columns
skater_games.columns = ['rank', 'date', 'game_num', 'age', 'team', 'home_away_status', 'opponent', 'result', 
                        'G', 'A', 'P', 'rating', 'PIM', 'EVG', 'PPG', 'SHG', 'GWG', 'EVA', 'PPA', 'SHA', 'S', 'S_perc',
                        'shifts', 'TOI', 'HIT', 'BLK', 'FOW', 'FOL', 'FOW_perc', 'player_id', 'season']

# Remove these weird rows... They are not actual data (just repeated column names)
skater_games = skater_games.loc[skater_games[('rank')] != 'Rk']

# Left join the player_name column to the data frame
skater_games = skater_games.merge(skater_ids_names, on = 'player_id', how = 'left')

# Reorder the columns 
skater_games = skater_games[['rank', 'player_id', 'player_name', 'age', 'season', 'game_num', 'date', 'team', 
                            'opponent', 'home_away_status', 'result', 'G', 'A', 'P', 'rating', 'PIM', 'EVG', 
                             'PPG', 'SHG', 'GWG', 'EVA', 'PPA', 'SHA', 'S', 'S_perc', 'shifts', 'TOI', 'HIT', 
                             'BLK', 'FOW', 'FOL', 'FOW_perc']]

# Clean home_away_status column. 1 means the player played a home game. 0 means the game was on the road.
skater_games = skater_games.replace({'home_away_status': {'@':0, np.nan:1}})

# Fix age strings to just reflect age in years. We probably don't care about days since last birthday.
skater_games['age'] = skater_games.age.str.extract(pat = '(\d+)')

# Convert time on ice (TOI) to minutes using decimal. For example 22:40 should become 22.666667.
skater_games[['min', 'sec']] = skater_games.TOI.str.split(':', expand = True)
skater_games['TOI'] = skater_games['min'].astype('float') + skater_games['sec'].astype('float') / 60

# Now drop columns we don't need.
skater_games = skater_games.drop(columns = ['rank', 'min', 'sec'])

# Convert team to 3 letter code
skater_games['team'] = skater_games['team'].str.lower().replace(team_name_dict)
skater_games['opponent'] = skater_games['opponent'].str.lower().replace(team_name_dict)

# Convert columns to correct types. The proper types are listed below.
    # Date --> [date]
    # Integer --> [game_num, age, home_away_status, G, A, P, rating, PIM, EVG, PPG, SHG, GWG, EVA, PPA, 
                # SHA, S, shifts, HIT, BLK, FOW, FOL, season]
    # Float --> [S_per, TOI, FOW_perc]
    # String --> [player_id, team, opponent, result]
skater_games = skater_games.astype({'date':'datetime64[ns]','game_num':'int', 'age':'int',
                                              'team':'str', 'home_away_status':'int', 'opponent':'str', 
                                              'result':'str', 'G':'int', 'A':'int', 'P':'int', 'rating':'int', 
                                              'PIM':'int', 'EVG':'int', 'PPG':'int', 'SHG':'int', 'GWG':'int', 
                                              'EVA':'int', 'PPA':'int', 'SHA':'int', 'S':'int', 'S_perc':'float',
                                              'shifts':'int', 'TOI':'float', 'HIT':'int', 'BLK':'int', 'FOW':'int', 
                                              'FOL':'int', 'FOW_perc':'float', 'player_id':'str', 'season':'int', 
                                              'player_name':'str'})



ValueError: cannot convert float NaN to integer: Error while type casting for column 'age'

## Clean data frame for goalies

In [64]:
# Rename columns
goalie_games.columns = ['rank', 'date', 'game_num', 'age', 'team', 'home_away_status', 'opponent', 'result', 
                        'decision', 'GA', 'SA', 'SV', 'SV_perc', 'shutout', 'PIM', 'TOI', 'player_id', 'season']

# Remove these weird rows... They are not actual data (just repeated column names)
goalie_games = goalie_games.loc[goalie_games[('rank')] != 'Rk']

# Left join the player_name column to the data frame
goalie_games = goalie_games.merge(goalie_ids_names, on = 'player_id', how = 'left')

# Reorder the columns 
goalie_games = goalie_games[['rank', 'player_id', 'player_name', 'age', 'season', 'game_num', 'date', 'team', 
                            'opponent', 'home_away_status', 'result', 'decision', 'GA', 'SA', 'SV', 'SV_perc', 
                             'shutout', 'PIM', 'TOI']]

# Clean home_away_status column. 1 means the player played a home game. 0 means the game was on the road.
goalie_games = goalie_games.replace({'home_away_status': {'@':0, np.nan:1}})

# Fix age strings to just reflect age in years. We probably don't care about days since last birthday.
goalie_games['age'] = goalie_games.age.str.extract(pat = '(\d+)')

# Convert time on ice (TOI) to minutes using decimal. For example 22:40 should become 22.666667.
goalie_games[['min', 'sec']] = goalie_games.TOI.str.split(':', expand = True)
goalie_games['TOI'] = goalie_games['min'].astype('float') + goalie_games['sec'].astype('float') / 60

# Now drop columns we don't need.
goalie_games = goalie_games.drop(columns = ['rank', 'min', 'sec'])

# Convert team to 3 letter code
goalie_games['team'] = goalie_games['team'].str.lower().replace(team_name_dict)
goalie_games['opponent'] = goalie_games['opponent'].str.lower().replace(team_name_dict)

# Convert columns to correct types. The proper types are listed below.
    # Date --> [date]
    # Integer --> [age, season, game_num, home_away_status, GA, SA, SV, shutout, PIM]
    # Float --> [SV_perc, TOI]
    # String --> [player_id, player_name, team, opponent, result, decision]
goalie_games = goalie_games.astype({'date':'datetime64[ns]', 'age':'int', 'season':'int', 'game_num':'int', 
                                    'home_away_status':'int', 'GA':'int', 'SA':'int', 'SV':'int', 
                                    'shutout':'int', 'PIM':'int', 'SV_perc':'float', 'TOI':'float', 
                                    'player_id':'str', 'player_name':'str', 'team':'str', 'opponent':'str', 
                                    'result':'str', 'decision':'str'})

ValueError: cannot convert float NaN to integer: Error while type casting for column 'age'

## Sanity checks for skaters

In [34]:
# Number of unique goalies
print(len(skater_games.player_id.unique()))

# All players should have between 1 and 82 games played (UNLESS THEY WERE TRADED AND NEVER MISSED A GAME)
print(sum(skater_games.groupby(['player_id'])['player_id'].count() <= 0))
print(sum(skater_games.groupby(['player_id'])['player_id'].count() > 82))

950
0
4


## Sanity checks for goalies

In [74]:
# Number of unique goalies
print(len(goalie_games.player_id.unique()))

# All goalies should have between 1 and 82 games played 
print(sum(goalie_games.groupby(['player_id'])['player_id'].count() <= 0))
print(sum(goalie_games.groupby(['player_id'])['player_id'].count() > 82))

107
0
0


In [75]:
# Goals against should equal shots against minus saves
print(sum(goalie_games.GA != goalie_games.SA - goalie_games.SV))

0


## Write the final data frame to a .csv file

In [76]:
# Save skaters
directory_to_save_s = '../data/stats/past_seasons/skaters_' + str(season) + '.csv'
skater_games.to_csv(directory_to_save_s, index = False)

# Save goalies
directory_to_save_g = '../data/stats/past_seasons/goalies_' + str(season) + '.csv'
goalie_games.to_csv(directory_to_save_g, index = False)